In [17]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import os
from glob import glob
import subprocess
import shutil
import traceback
import random

from concurrent.futures import ProcessPoolExecutor

from backports import tempfile

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import psrchive

import pipe

In [19]:
data_location = "/nimrod1/aarchiba/0337+17"
data_location_name = data_location+"/{telescope}/{epoch}"

In [20]:
wsrt_data = "/nimrod1/aarchiba/0337+17/WSRT/"
if not os.path.exists(wsrt_data):
    os.mkdir(wsrt_data)

In [21]:
wsrt_raw_location = "/nimrod1/aarchiba/0337+17/WSRT-raw"
wsrt_obs_glob = wsrt_raw_location+"/*/*0337*/"

class DiscontinuityError(ValueError):
    pass

def process_wsrt_day(d):
    success_file = d+"/success"
    if os.path.exists(success_file):
        return open(success_file,"rt").read().strip()
    with tempfile.TemporaryDirectory() as td:
        for s in glob(d+"/Band*"):
            b = s.split("/")[-1]
            subprocess.check_call(["psradd", "-o",td+"/"+b+".tadd.ar"]+glob(s+"/u*.ar"))
        subprocess.check_call(["psradd", "-o", td+"/raw.ar", "-R"]+glob(td+"/Band*.tadd.ar"))
        shutil.copy(td+"/raw.ar", "raw.ar")
        F = psrchive.Archive_load(td+"/raw.ar")
        l = (F.end_time()-F.start_time()).in_seconds()
        M = "%.1f" % (((F.end_time()+F.start_time())).in_days()/2)
        #T = F.get_nsubint()*F.get_first_Integration().get_duration()
        T = F.integration_length()
        if np.abs(l-T)> 5:
            raise DiscontinuityError("Observation in directory %s appears not to be contiguous: "
                             "total integration time %f but start-to-end time %f"
                             % (d,T,l))
        shutil.copy(td+"/raw.ar", wsrt_data+"wsrt_%s.raw.ar" % M)
        with open(success_file,"wt") as f:
            f.write(M)
        return M

In [ ]:
g = list(glob(wsrt_obs_glob))
random.shuffle(g)
for d in g:
    print d
    try:
        print "success;", process_wsrt_day(d)
    except subprocess.CalledProcessError:
        print "psradd failure"
    except DiscontinuityError:
        traceback.print_exc()

In [6]:
def do_one(d):
    try:
        return process_wsrt_day(d)
    except subprocess.CalledProcessError:
        return "psradd problem with %s" % d
    except DiscontinuityError:
        return "discontinu
with ProcessPoolExecutor() as P:
    list(P.map(do_one,glob(wsrt_obs_glob)))

KeyboardInterrupt: 

In [ ]:
F = psrchive.Archive_load("raw.ar")

In [36]:
m = (F.end_time()-F.start_time()).in_seconds()

In [35]:
m.in_seconds()

769.9974296889412

In [37]:
F.get_bandwidth()

160.0

In [38]:
F.get_ephemeris()

<psrchive.Parameters; proxy of <Swig Object of type 'Pulsar::Parameters *' at 0x3eeddb0> >

In [39]:
F.get_nsubint()

77

In [41]:
I = F.get_first_Integration()

In [42]:
I.get_duration()

9.99993600000005

In [44]:
F.integration_length()

769.9950720000038

In [45]:
F.get_centre_frequency()

1380.0

In [12]:
ProcessPoolExecutor.map?

In [10]:
!ls

pipe.py  pipe.py~  pipe.pyc  raw.ar  summarize-observations.ipynb
